# Capstone Project: The Battle of the Neighbourhoods (Week 1)
### Corurse 9: Applied Data Science Capstone by IBM/Coursera

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)

## Introduction: Business Problem <a name="introduction"></a>

In this project, we will try to build a model to find the optimal neighbourhood for openning a new business. As an example we will specify the business type to be an **Italian restaurant**.

Since London is huge city with a lot of restraunts, we will try to identify the optimal neighbourhood to open a new Italian restaurant. We will try to identify the optimal neighborhood based on:
* Less number of **high rating** Italian restraunts.
* Less number of Italian restraunts.
* Less number of restraunts.


## Data <a name="data"></a>

Based on teh description of the [Business Problem](#introduction), we will need to get data of:
* List of neighbourhoods in London with their latitudes, longitudes and area.
* Number of restraunts (any type) in each neighbourhood.
* Number of Italian restraunts in the neighbourhood.
* Rating of each Italian restraunt in the neighbourhood.

We will extract the required data as follows:
* We will get the list of neighbourhoods in London and their geo-location from [Wikipedia page](#https://en.wikipedia.org/wiki/List_of_London_boroughs).
* We will get the restraunts information for each neighbourhood from **Foursquare API**.
* Also, for the map visualization we will get the location of London using **geopy API**.

#### Assumptions

First, lets import the required libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import re #Library to read config file

import geocoder

import folium

#!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim

import configparser #Library to read config file

import json

import requests

import math

### Neighbourhood Data
We will retrieve the list of London neighbourhoods from Wikipedia page [List of London boroughs](#https://en.wikipedia.org/wiki/List_of_London_boroughs) which contain London neighbourhoods and their Geo-Location information and other irrelavant information which will be excluded.

In [2]:
# Funtion to calculate area in meter square from mile square
def convert_mi2_m2(row) :
    mi_2 = row["Area (sq mi)"]
    m_2 = mi_2 * 2590000
    return m_2

# function to calculate Circle radius in meteres from meter square area
def convert_area_radius(row) :
    area = row["Area"]
    radius = math.sqrt((area / math.pi))
    return radius

# read html tables from the Wikipedia page
dfs = pd.read_html('https://en.wikipedia.org/wiki/List_of_London_boroughs') 
print('List of London boroughs loaded')

# we get 2 dataframes so we concat them into one dataframe 
df = pd.concat([dfs[0], dfs[1]])

# Caclculate Area and radius in metric measurements
df["Area"] = df.apply(lambda row : convert_mi2_m2(row), axis=1)
df["Area-Radius"] = df.apply(lambda row : convert_area_radius(row), axis=1)

# extract only the columns we are interested in
london_df = df[['Borough', 'Co-ordinates', 'Area', 'Area-Radius']].reset_index(drop=True)
london_df.rename(columns={'Borough' : 'Neighbourhood'}, inplace = True)

# clean Borough from [note 1]
note_regex = re.compile(r"\[note \d]")
london_df['Neighbourhood'] = london_df['Neighbourhood'].str.replace(pat=note_regex, repl='', regex=True)

#Split the Co-ordinates column into 2 columns for DMS and Decimal geo-formats
london_df[['Co-ordinates_DMS', 'Co-ordinates_DEC']] = london_df['Co-ordinates'].str.split(pat=' / ', expand=True)
london_df[['Latitude', 'Longitude']] = london_df['Co-ordinates_DEC'].str.split(expand=True)

# extract and clean Latitude and Longitude from Co-ordinates_DEC
def clean_dec(dec_str):
    
    sign = -1 if re.search('[swSW]', dec_str) else 1
    dec_str = re.sub(r'°.', '', dec_str)
    dec_str = re.sub(' ', '', dec_str)
    dec_str = re.sub(u'\ufeff', '', dec_str)
    
    return sign * (float(dec_str))

london_df['Latitude'] = london_df['Latitude'].apply(clean_dec)
london_df['Longitude'] = london_df['Longitude'].apply(clean_dec)

# extract only needed columns
london_df = london_df[['Neighbourhood', 'Latitude', 'Longitude', 'Area', 'Area-Radius']]

london_df.head()


List of London boroughs loaded


,Neighbourhood,Latitude,Longitude,Area,Area-Radius
0,Barking and Dagenham,51.5607,0.1557,36078700.0,3388.835625
1,Barnet,51.6252,-0.1517,86739100.0,5254.513588
2,Bexley,51.4549,0.1505,60554200.0,4390.330342
3,Brent,51.5588,-0.2817,43253000.0,3710.506368
4,Bromley,51.4039,0.0198,150142300.0,6913.159800


Each neighborhood in London has different area. We already consider/assume the Longitude and Latitude we got from Wikipedia to be the center of each neighborhood. 

As we are going to provide the **radius** in our Foursquare APIs calls, we associated each neighborhood to a radius value of a circle whose area equals the neighborhood area.  

### London City Location <a name="london_loc"></a>

Now, le's get the location of London using **geopy API**.

In [3]:
def get_geo_location(addr) :
    geolocator = Nominatim(user_agent="uk_explorer")
    location = geolocator.geocode(addr)
    latitude = location.latitude
    longitude = location.longitude
    #print('The geograpical coordinate of London City are {}, {}.'.format(latitude, longitude))
    return [latitude, longitude]

address = 'London, UK'
london_location = get_geo_location(address)
print('Coordinate of {}: {}'.format(address, london_location))

Coordinate of London, UK: [51.5073219, -0.1276474]


Now we can visualize the London maps and its neighborhood. Each neighborhood center will be surrounded with circls of radius proportional to its area.

In [4]:
map_london = folium.Map(location=london_location, zoom_start=10)


folium.CircleMarker(
    london_location,
    radius=3,
    color='red',
    popup=address,
    fill=True,
    fill_color='red',
    fill_opacity=0.6
).add_to(map_london)

for lat, lng, radius, label in zip(london_df.Latitude, london_df.Longitude, london_df['Area-Radius'] , london_df.Neighbourhood) :
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(map_london)

    folium.Circle(
        [lat, lng],
        radius=radius,
        color='Yellow'
    ).add_to(map_london)

map_london

The Circle coverage of London doesn't seem to be good enough. Let's increase the radius of all neighborhoods with 30% and check again

In [5]:
london_df['Area-Radius'] *= 1.3

map_london = folium.Map(location=london_location, zoom_start=10)


folium.CircleMarker(
    london_location,
    radius=3,
    color='red',
    popup=address,
    fill=True,
    fill_color='red',
    fill_opacity=0.6
).add_to(map_london)

for lat, lng, radius, label in zip(london_df.Latitude, london_df.Longitude, london_df['Area-Radius'] , london_df.Neighbourhood) :
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(map_london)

    folium.Circle(
        [lat, lng],
        radius=radius,
        color='Yellow'
    ).add_to(map_london)

map_london


Now we have better coverage to London. We can use this dataset for our analysis.

### Restraunts Data

Now we need to build a data set of restaurants in each neighborhood. Our focus will be in restaurants in general and specially italian restaurants. For this purpose we will use **Foursquare APIs**.

#### Foursquare

First, let's prepare the basic **Foursquare** connection configurations.

In [6]:
# Read Foursquare authentication info from hidden config file
config = configparser.ConfigParser()
config.read('secrets.cfg')
CLIENT_ID = config['4square_personal']['CLIENT_ID']
CLIENT_SECRET = config['4square_personal']['CLIENT_SECRET']
VERSION = '20201103' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

REQUEST_DEFAULT_PARAMS = dict(
    client_id = CLIENT_ID,
    client_secret = CLIENT_SECRET,
    v=VERSION
)

We will use the **explore** endpoint to query for restaurants in each neighborhood.  
To limit the query result, we will specify parameter **section** as **food**. This should limit the result to food related venues. Also, we will utilize the **sortByPopularity** parameter to get results sorted based on Popularity.

In [7]:
# validate whether the venue category is a competitor restaurant and if it is Italian restaurant
def validate_rest_category(rest_cat_name) :
    excluded_categories = ['Café', 'Pub', 'Food Court', 'Fast Food', 'Shop', 'Coffee', 'Bakery', 'Breakfast']
    italian_list = ['Italian Restaurant', 'Abruzzo Restaurant', 'Agriturismo', 'Aosta Restaurant', 'Basilicata Restaurant', 'Calabria Restaurant', 'Campanian Restaurant', 'Emilia Restaurant', 'Friuli Restaurant', 'Ligurian Restaurant', 'Lombard Restaurant', 'Malga', 'Marche Restaurant', 'Piadineria', 'Piedmontese Restaurant', 'Puglia Restaurant', 'Romagna Restaurant', 'Roman Restaurant', 'Sardinian Restaurant', 'Sicilian Restaurant', 'South Tyrolean Restaurant', 'Trattoria/Osteria', 'Trentino Restaurant', 'Tuscan Restaurant', 'Umbrian Restaurant', 'Veneto Restaurant']
    is_restaurant = True
    for excl in excluded_categories :
        if excl in rest_cat_name :
            is_restaurant = False
            break
    is_italian = rest_cat_name in italian_list

    return is_restaurant, is_italian


def get_neighborhood_restaurants(lat, lng, radius) :
#def get_neighborhood_restaurants(venue) :
    URL_4SQU_EXPLORE = 'https://api.foursquare.com/v2/venues/explore'
    category_lst = []

    explore_params = REQUEST_DEFAULT_PARAMS.copy()

    explore_params.update({
        "ll" : '{}, {}'.format(lat, lng), 
        "radius" : radius,
        "section" : 'food',
        #"query" : 'restaurant',
        "sortByPopularity" : 1
        })
    restaurants_list = requests.get(url=URL_4SQU_EXPLORE, params=explore_params).json()['response']['groups'][0]['items']
    return restaurants_list

#london_df[['Neighbourhood' == 'Barnet']]
restaurants_df = pd.DataFrame(columns=['Neighbourhood', 'Restaurant_ID', 'Restaurant_Name', 'Category_ID', 'Category_Name', 'Restaurant_Latitude', 'Restaurant_Longitude', 'Is_Italian'])
for lat, lng, radius, neighbourhood in zip(london_df.Latitude, london_df.Longitude, london_df['Area-Radius'] , london_df.Neighbourhood) :
    rest_lst = get_neighborhood_restaurants(lat, lng, radius)
    for rest_info in rest_lst :
        restaurants_id = rest_info['venue']['id']
        restaurants_name = rest_info['venue']['name']
        rest_cat_id = rest_info['venue']['categories'][0]['id']
        rest_cat_name = rest_info['venue']['categories'][0]['name']
        rest_lat = rest_info['venue']['location']['lat']
        rest_lng = rest_info['venue']['location']['lng']
        is_restaurant, is_italian = validate_rest_category(rest_cat_name)
        if is_restaurant :
            restaurants_df = restaurants_df.append({'Neighbourhood' : neighbourhood, 'Restaurant_ID' : restaurants_id, 'Restaurant_Name' : restaurants_name, 'Category_ID' : rest_cat_id, 'Category_Name' : rest_cat_name, 'Restaurant_Latitude' : rest_lat, 'Restaurant_Longitude' : rest_lng, 'Is_Italian' : is_italian}, ignore_index=True)


restaurants_df.head()


,Neighbourhood,Restaurant_ID,Restaurant_Name,Category_ID,Category_Name,Restaurant_Latitude,Restaurant_Longitude,Is_Italian
0,Barking and Dagenham,4bb9e6183db7b713faa0229a,Moby Dick,4bf58dd8d48988d1c4941735,Restaurant,51.580585,0.140618,False
1,Barking and Dagenham,4e7a1f5414954a343fb58258,The Pipe Major,4bf58dd8d48988d1c4941735,Restaurant,51.545795,0.165834,False
2,Barking and Dagenham,50baf653183fee49a8b0b4cc,Lara Grill,4f04af1f2fb6e1c99f3db0bb,Turkish Restaurant,51.562445,0.147178,False
3,Barking and Dagenham,4eb064fb775bbddde7b4f546,Cosmo,4bf58dd8d48988d1c4941735,Restaurant,51.575445,0.180456,False
4,Barking and Dagenham,4c3218fa3896e21e07f2e790,The Compasses (Harvester),52e81612bcbc57f1066b7a05,English Restaurant,51.557806,0.209121,False


In [8]:
london_df = london_df.merge(restaurants_df, on='Neighbourhood', how='inner')

london_df.head()

,Neighbourhood,Latitude,Longitude,Area,Area-Radius,Restaurant_ID,Restaurant_Name,Category_ID,Category_Name,Restaurant_Latitude,Restaurant_Longitude,Is_Italian
0,Barking and Dagenham,51.5607,0.1557,36078700.0,4405.486312,4bb9e6183db7b713faa0229a,Moby Dick,4bf58dd8d48988d1c4941735,Restaurant,51.580585,0.140618,False
1,Barking and Dagenham,51.5607,0.1557,36078700.0,4405.486312,4e7a1f5414954a343fb58258,The Pipe Major,4bf58dd8d48988d1c4941735,Restaurant,51.545795,0.165834,False
2,Barking and Dagenham,51.5607,0.1557,36078700.0,4405.486312,50baf653183fee49a8b0b4cc,Lara Grill,4f04af1f2fb6e1c99f3db0bb,Turkish Restaurant,51.562445,0.147178,False
3,Barking and Dagenham,51.5607,0.1557,36078700.0,4405.486312,4eb064fb775bbddde7b4f546,Cosmo,4bf58dd8d48988d1c4941735,Restaurant,51.575445,0.180456,False
4,Barking and Dagenham,51.5607,0.1557,36078700.0,4405.486312,4c3218fa3896e21e07f2e790,The Compasses (Harvester),52e81612bcbc57f1066b7a05,English Restaurant,51.557806,0.209121,False


In [9]:
map_london = folium.Map(location=london_location, zoom_start=11)

folium.CircleMarker(
    london_location,
    radius=3,
    color='red',
    popup=address,
    fill=True,
    fill_color='red',
    fill_opacity=0.6
).add_to(map_london)

for lat, lng, radius, label, special in zip(london_df.Restaurant_Latitude, london_df.Longitude, london_df.Restaurant_Longitude , london_df.Restaurant_Name, london_df.Is_Italian) :
    color = 'blue'
    if special :
        color = 'green'

    folium.CircleMarker(
        [lat, lng],
        radius=2,
        color=color,
        popup=label,
        fill = True,
        fill_color= color,
        fill_opacity=0.6
    ).add_to(map_london)

map_london

This conclude our Data Section.  
Now we have the list of restaurants and their locations. Also, we know which restaurant is Italian or not.